In [1]:
!pip install -q chonkie docling model2vec rich torch transformers tqdm requests


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install /Users/ln/dev/helix-py

Processing /Users/ln/dev/helix-py
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for helix-py: filename=helix_py-0.1.0-py3-none-any.whl size=18683 sha256=20a3e00cd6c95113f14792c8319a5088d628dbe07fc73de5460bd0681f0bf41e
  Stored in directory: /Users/ln/Library/Caches/pip/wheels/f9/37/7c/f0015793ba184eb4a2298b48b11a3bad96c63014471e2a40f2
Successfully built helix-py

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [14]:
from chonkie import RecursiveChunker, RecursiveRules, RecursiveLevel
from docling.document_converter import DocumentConverter
from transformers import AutoTokenizer, AutoModel
from rich.console import Console
from rich.text import Text
from typing import List, Tuple
import numpy as np
import os
import torch
from tqdm import tqdm
import requests

import helix
from helix.client import Query
from helix.types import Payload

In [4]:
console = Console()

# A wrapper to pretty print
def rprint(text: str, console: Console=console, width: int = 80) -> None:
  richtext = Text(text)
  console.print(richtext.wrap(console, width=width))

In [5]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

In [6]:
helix_docs_endpoints = [
        "https://docs.helix-db.com/info/hql",
        "https://docs.helix-db.com/introduction/cookbook/basic",
        "https://docs.helix-db.com/get-started/installation",
        "https://docs.helix-db.com/get-started/sql-ingestion"
        "https://docs.helix-db.com/hql/schema-definition",
        "https://docs.helix-db.com/hql/query-structure",
        "https://docs.helix-db.com/hql/source/source",
        "https://docs.helix-db.com/hql/source/adding",
        "https://docs.helix-db.com/hql/steps/traversals/steps_nodes",
        "https://docs.helix-db.com/hql/steps/traversals/steps_edges",
        "https://docs.helix-db.com/hql/steps/conditions",
        "https://docs.helix-db.com/hql/steps/anonymous",
        "https://docs.helix-db.com/hql/steps/properties/property-access",
        "https://docs.helix-db.com/hql/steps/properties/property-additions",
        "https://docs.helix-db.com/hql/steps/properties/property-exclusion",
        "https://docs.helix-db.com/hql/steps/properties/property-remappings",
        "https://docs.helix-db.com/hql/steps/deleting",
        "https://docs.helix-db.com/hql/steps/updating",
        "https://docs.helix-db.com/hql/steps/operations",
        "https://docs.helix-db.com/hql/vectors/inserting",
        "https://docs.helix-db.com/hql/vectors/searching",
        "https://docs.helix-db.com/hql/types"
]

In [49]:
def filter_menu_sections(text):
    lines = text.splitlines()
    cleaned_lines = []
    skip_count = 0
    i = 0

    while i < len(lines):
        line = lines[i]
        # Check if we’re in a skip block
        if skip_count > 0:
            skip_count -= 1
            i += 1
            continue
        # Check for the target headers
        if line.startswith('##### Getting Started') or line.startswith('##### HelixQL'):
            # Skip this line and the next 17 lines
            skip_count = 17
            i += 1
            continue
        # Keep the line if not skipping
        cleaned_lines.append(line)
        i += 1

    return '\n'.join(cleaned_lines)

In [50]:
converter = DocumentConverter()
results = [converter.convert(doc) for doc in helix_docs_endpoints]
text_results = [res.document.export_to_markdown() for res in results]
text = "\n".join(text_results)
text = filter_menu_sections(text)

In [61]:
rules = RecursiveRules(
    levels=[
        RecursiveLevel(delimiters=['######', '#####', '####', '###', '##', '#']),
        RecursiveLevel(delimiters=['\n\n', '\n', '\r\n', '\r']),
        RecursiveLevel(delimiters='.?!;:'),
        RecursiveLevel()
    ]
)
chunker = RecursiveChunker(rules=rules, chunk_size=300)

In [62]:
chunks = chunker(text)
print(f"Total number of chunks: {len(chunks)}")
# @title A quick look at our chunks~
for chunk in chunks[:4]:
    rprint(chunk.text)
    print('-'*80, '\n\n')

Total number of chunks: 45


# What is a HQL?

Helix Query Language (HQL) is a powerful graph traversal and similarity search 
query language

## ​Documentation

## Schema Definition

Define and understand HQL schemas for your data

## Query Structure

Learn the basic structure and syntax of HQL queries

## Graph Traversals

Master graph traversal operations in HQL

## Vector Operations

Explore how to use vectors in HelixQL

## Property Access

Learn how to access and manipulate data properties

## HelixQL Types

Learn about the different types of data in HelixQL

Was this page helpful?

- Documentation

# A basic example

A basic example of how to use HelixDB

We’re going to create a simple social network where users can follow each other 
and create posts.

#

-------------------------------------------------------------------------------- 




## ​Step 1: Define the schema in schema.hx

We’re going to define the schema for our social network in schema.hx.
We’re going to have User nodes where users can Follow other users.
We’re also going to have Post nodes where users can Create posts.

```
N::User {
    name: String,
    age: U32,
    email: String,
    created_at: I32,
    updated_at: I32,
}

N::Post {
    content: String,
    created_at: I32,
    updated_at: I32,
}

E::Follows {
    From: User,
    To: User,
    Properties: {
        since: I32,
    }
}

E::Created {
    From: User,
    To: Post,
    Properties: {
        created_at: I32,
    }
}
```

#

-------------------------------------------------------------------------------- 




## ​Step 2: Inserting data

Creating a user is done by inserting a new node into the graph.

```
QUERY CreateUser(name: String, age: U32, email: String, now: I32) =>
    user <- AddN<User>({name, age, email, created_at: now, updated_at: now})
    RETURN user
```

To let users follow another user, we need to create an edge between them.

```
QUERY CreateFollow(follower_id: String, followed_id: String, now: I32) =>
    follower <- N<User>(follower_id)
    followed <- N<User>(followed_id)
    AddE<Follows>({since: now})::From(follower)::To(followed)
    RETURN "success"
```

To create a post, we need to create a post node and an edge between the user and
the post.

```
QUERY CreatePost(user_id: String, content: String, now: I32) =>
    user <- N<User>(user_id)

-------------------------------------------------------------------------------- 




post <- AddN<Post>({content, created_at: now, updated_at: now})
    AddE<Created>({created_at: now})::From(user)::To(post)
    RETURN post
```

#

-------------------------------------------------------------------------------- 




In [63]:
def vectorize_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().tolist()
    return embedding

def vectorize_chunked(chunked: List[str]) -> List[List[float]]:
    # embedding dims: 768
    vectorized = []
    for chunk in tqdm(chunked):
        embedding = vectorize_text(chunk)
        vectorized.append(embedding)
    return vectorized

OLLAMA_API_URL = "http://localhost:11434/api/generate"

def get_ollama_response(prompt):
    payload = {
        #"model": "deepseek-r1:7b",
        "model": "llama3.1:8b",
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        raise Exception(f"Ollama API request failed with status {response.status_code}")

In [64]:
items = [chunk.text for chunk in chunks]
vectors = vectorize_chunked(items)
print(f"doc length: {len(text)} chars, num of vectors: {len(vectors)}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:02<00:00, 15.61it/s]

doc length: 27538 chars, num of vectors: 45


In [131]:
# A simple function to make LLM prompts with chunks
def create_prompt(chunks: List[str], query: str) -> str:
  prompt_template = """<instructions>
  Based on the provided contexts, answer the given question to the best of your ability. Remember to also add citations at appropriate points in the format of square brackets like [1][2][3], especially at sentence or paragraph endings.
  You will be given a couple passages in the context, marked with a label 'Doc [1]:' to denote the passage number. Use that number for citations."
  </instructions>

  <context>
  {context}
  </context>

  <query>
  {query}
  </query>
  """
  context = "\n\n".join([f"{chunk}" for chunk in chunks])
  prompt = prompt_template.format(context=context, query=query)
  #print(prompt)
  return prompt

In [107]:
db = helix.Client(local=True)

class ragloaddocs(Query):
    def __init__(self, docs: List[Tuple[str, List[Tuple[List[float], str]]]]):
        super().__init__()
        self.docs = docs

    def query(self) -> List[Payload]: # TODO: batch send
        docs_payload = []
        for doc, vectors in self.docs:
            docs_payload.append({ "doc": doc, "vectors": [{ "vec": vec, "chunk": chunk } for vec, chunk in vectors]})

        return [{ "docs": docs_payload }]

    def response(self, response):
        return response

class ragsearchdocs(Query):
    def __init__(self, query_vector: List[float], k: int=4):
        super().__init__()
        self.query_vector = query_vector
        self.k = k

    def query(self) -> List[Payload]:
        return [{ "query": self.query_vector, "k": self.k }]

    def response(self, response):
        return response.get("chunks")

[HELIX] Helix instance found at 'http://0.0.0.0:6969'


In [67]:
# insert all the docs to vectors into helix
db.query(ragloaddocs([(text, list(zip(vectors, items)))]))

[HELIX] Querying 'http://0.0.0.0:6969/ragloaddocs': 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.61it/s]


[{'message': 'Success'}]

In [133]:
user_prompt = "Explain Traversals from Nodes. I want to get the nodes connected by outgoing edges of specific type."
query_embedding = vectorize_text(user_prompt)
res = db.query(ragsearchdocs(query_embedding, 6))[0]
response = get_ollama_response(create_prompt([f"Doc[{i+1}]: {text['content']}" for i, text in enumerate(res)], user_prompt))
print(f"reponse: {response}")

[HELIX] Querying 'http://0.0.0.0:6969/ragsearchdocs': 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 139.01it/s]


reponse: Based on the provided context, I can answer your question as follows:

To explain traversals from nodes and get the nodes connected by outgoing edges of a specific type, you can use the `Out` and `OfType` operations in HelixQL.

Here's an example query that demonstrates how to achieve this:

```
QUERY GetFollowers(followed_id: String) =>
    followed <- N<User>(followed_id)
    followers <- Out<Follows>()::From(followed)::To()
    RETURN followers
```

In this query, we start from a node of type `User` with the ID `followed_id`. Then, we use the `Out` operation to traverse outwards from that node and get all edges of type `Follows`. The `::To()` part specifies that we want to follow these edges to their destinations. Finally, we return the list of followers.

This query will give you all nodes connected by outgoing edges of type `Follows` from the specified `followed_id`.

Please note that this is just an example based on the provided context, and you should adjust it accordin